In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
import plotly.express as px

/tmp/ipykernel_63430/436934381.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-12 21:22:26.989921: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-12 21:22:27.104303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 21:22:27.104367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin 

In [2]:
#model = tf.keras.models.load_model('predict_model.keras')
#model = pickle.load(open('linear_regressor.pkl', 'rb'))

In [3]:
scaler = pickle.load(open('scaler.pkl', 'rb'))

In [4]:
columns = ['CO(mg/m^3)_initial', 'Volume(m^3)', 'N_people_MA_prev_10min',	'Ambient-Air-Pump(L/min)', 'Ambient-Air-Pump_power(%)',	'Ambient-Air-Pump_number', 'CO(mg/m^3)_final']
#mae: 0.1278

In [5]:
output_column = 'CO(mg/m^3)_final'

In [23]:
input_columns = ['CO(mg/m^3)_initial', 'Volume(m^3)', 'N_people_MA_prev_10min',	'Ambient-Air-Pump(L/min)', 'Ambient-Air-Pump_power(%)',	'Ambient-Air-Pump_number']

In [24]:
CO = 0.018
volume = 30
people_now = 10
people_tn = [people_now, people_now, people_now, people_now, people_now, people_now, people_now, people_now, people_now, people_now]
pumps_l_min = 566
pump_power = 0.0
n_pumps = 8
index = 0

In [25]:
def create_df_input(CO, volume, people_tn, pumps_l_min, pump_power, n_pumps, index):
    df = pd.DataFrame({
        'CO(mg/m^3)_initial':CO,
        'Volume(m^3)':volume, 
        'N_people_MA_prev_10min': sum(people_tn) / len(people_tn),	
        'Ambient-Air-Pump(L/min)': pumps_l_min, 
        'Ambient-Air-Pump_power(%)': pump_power,
        'Ambient-Air-Pump_number': n_pumps
    }, index=[index])
    return df

In [26]:
df_sim = pd.DataFrame(columns = columns)
for i in range(10):
    index = i
    df_t = create_df_input(CO, volume, people_tn, pumps_l_min, pump_power, n_pumps, index)
    df_t_scaled = scaler.transform(df_t)
    df_t_scaled = pd.DataFrame(df_t_scaled, columns=input_columns)
    CO = model.predict(df_t_scaled)[0][0]
    CO += (np.random.rand(1)[0]*2 -1)* 0.01 * CO
    df_t["CO(mg/m^3)_final"] = CO
    df_sim = pd.concat([df_sim,df_t])

/tmp/ipykernel_63430/235172288.py:10: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [27]:
fig = px.line(df_sim["CO(mg/m^3)_final"])
fig.show()

In [28]:
df_sim

,CO(mg/m^3)_initial,Volume(m^3),N_people_MA_prev_10min,Ambient-Air-Pump(L/min),Ambient-Air-Pump_power(%),Ambient-Air-Pump_number,CO(mg/m^3)_final
0,0.018000,30,10.0,566,0.0,8,0.901194
1,0.901194,30,10.0,566,0.0,8,0.981617
2,0.981617,30,10.0,566,0.0,8,0.976032
3,0.976032,30,10.0,566,0.0,8,0.976114
4,0.976114,30,10.0,566,0.0,8,0.983268
5,0.983268,30,10.0,566,0.0,8,0.987935
6,0.987935,30,10.0,566,0.0,8,0.977049
7,0.977049,30,10.0,566,0.0,8,0.988269
8,0.988269,30,10.0,566,0.0,8,0.989088
9,0.989088,30,10.0,566,0.0,8,0.982650
